Data downloaded from:

https://catalog.data.gov/dataset/lottery-mega-millions-winning-numbers-beginning-2002

### Set Up

In [72]:
import pandas as pd
import numpy as np
import datetime

mega_raw = pd.read_csv('Lottery_Mega_Millions_Winning_Numbers__Beginning_2002.csv', encoding="Latin-1")
mega_raw = mega_raw.drop(columns=['Multiplier'])

#print(mega_raw[mega_raw['Draw Date']=='10/31/2017'].index.values)

#versions changes (Date - White Ball/Mega Ball):

# 05/15/2002 -  52/52 
# 06/24/2005 -  56/46 - index 324
# 10/22/2013 -  75/15 - index 1190
# 10/31/2017 -  70/25  - index 1610

mega_first = mega_raw.loc[:323, :]
mega_second = mega_raw.loc[324:1189, :]
mega_third = mega_raw.loc[1190:1609, :]
mega_fourth = mega_raw.loc[1610:, :]

print('First dataframe size:', mega_first.count(), sep='\n')
print('\n', 'Second dataframe size:', mega_second.count(), sep='\n')
print('\n', 'Third dataframe size:', mega_third.count(), sep='\n')
print('\n', 'Fourth dataframe size:', mega_fourth.count(), sep='\n')



First dataframe size:
Draw Date          324
Winning Numbers    324
Mega Ball          324
dtype: int64


Second dataframe size:
Draw Date          866
Winning Numbers    866
Mega Ball          866
dtype: int64


Third dataframe size:
Draw Date          420
Winning Numbers    420
Mega Ball          420
dtype: int64


Fourth dataframe size:
Draw Date          262
Winning Numbers    262
Mega Ball          262
dtype: int64


### Lottery Frequency and Date Functions

In [8]:
# This function will split the winning numbers column into columns each with only one number
def split_cols(df, name1, name2, name3, name4, name5):
    df[[name1, name2, name3, name4, name5]] = df['Winning Numbers'].str.split(' ', expand = True).astype(int) 
    df= df.drop(columns=['Winning Numbers'])
    return df   

# This function will return dataframe on Mega Ball only
def megaball(df, col_number):
    df = df.iloc[:,:col_number]
    return df

# This function will return dataframe on White Ball only
def whiteball(df, ball_type):
    df = df.drop(columns='Mega Ball',)
    df = pd.melt(df, id_vars=['Draw Date'], var_name='Ball Order', value_name=ball_type)
    df = df.drop(columns = ['Ball Order'])
    return df

# This function will return a frequency count for each number
def ballcount(df, col_name, count_name):
    df = df[col_name].value_counts().rename_axis(col_name).reset_index(name=count_name)
    return df
    
# This function would return the last drawn date for each number
def checkdate(df):
    emptydict = {}
    for i, j in df.iterrows():
        date = j[0]
        number = j[1]
        emptydict[number] = date
    return emptydict


def datelastdrawn(df1, df2, ball_type, date):
    # Merging the frequency dataframe with date dataframe
    df3 = df1.merge(df2, left_on = ball_type, right_on = ball_type, how= 'right')
    df3['Date']= pd.to_datetime(df3['Date'])
    
    # Calculating days since the number has been last drawn
    if date == 'Now':
        now = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        df3['Days Since Last Drawing'] = now - df3['Date']
    else:
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        df3['Days Since Last Drawing'] = date - df3['Date']
    return df3

### Mega Millions Frequency 05/17/2002 - 06/21/2005

In [80]:
first_df = mega_first.copy()

firstdf = split_cols(first_df, 'First', 'first', 'first', 'first', 'Fifth')
first_mega = megaball(firstdf, 2)
first_white = whiteball(firstdf, 'White Ball')
first_megacount = ballcount(first_mega, 'Mega Ball', 'Mega Count')
first_whitecount = ballcount(first_white, 'White Ball', 'White Count')

first_whitedate = pd.DataFrame(checkdate(first_white).items(), columns=['White Ball', 'Date'])
first_megadate = pd.DataFrame(checkdate(first_mega).items(), columns=['Mega Ball', 'Date'])

lastdate = firstdf['Draw Date'].iloc[-1]
first_whitecountdate = datelastdrawn(first_whitecount, first_whitedate, 'White Ball', lastdate)
first_megacountdate = datelastdrawn(first_megacount, first_megadate, 'Mega Ball', lastdate)

first_mega_sorted = first_megacountdate.sort_values(by = 'Days Since Last Drawing')
first_white_sorted = first_whitecountdate.sort_values(by = 'Days Since Last Drawing')

### Mega Millions Frequency 06/24/2005 - 10/15/2013

In [77]:
second_df = mega_second.copy()

seconddf = split_cols(second_df, 'First', 'Second', 'second', 'second', 'Fifth')
second_mega = megaball(seconddf, 2)
second_white = whiteball(seconddf, 'White Ball')
second_megacount = ballcount(second_mega, 'Mega Ball', 'Mega Count')
second_whitecount = ballcount(second_white, 'White Ball', 'White Count')

second_whitedate = pd.DataFrame(checkdate(second_white).items(), columns=['White Ball', 'Date'])
second_megadate = pd.DataFrame(checkdate(second_mega).items(), columns=['Mega Ball', 'Date'])

lastdate = seconddf['Draw Date'].iloc[-1]
second_whitecountdate = datelastdrawn(second_whitecount, second_whitedate, 'White Ball', lastdate)
second_megacountdate = datelastdrawn(second_megacount, second_megadate, 'Mega Ball', lastdate)

second_mega_sorted = second_megacountdate.sort_values(by = 'Days Since Last Drawing')
second_white_sorted = second_whitecountdate.sort_values(by = 'Days Since Last Drawing')

### Mega Millions Frequency 10/22/2013 - 10/27/2017

In [78]:
third_df = mega_third.copy()

thirddf = split_cols(third_df, 'First', 'third', 'third', 'third', 'Fifth')
third_mega = megaball(thirddf, 2)
third_white = whiteball(thirddf, 'White Ball')
third_megacount = ballcount(third_mega, 'Mega Ball', 'Mega Count')
third_whitecount = ballcount(third_white, 'White Ball', 'White Count')

third_whitedate = pd.DataFrame(checkdate(third_white).items(), columns=['White Ball', 'Date'])
third_megadate = pd.DataFrame(checkdate(third_mega).items(), columns=['Mega Ball', 'Date'])

lastdate = thirddf['Draw Date'].iloc[-1]
third_whitecountdate = datelastdrawn(third_whitecount, third_whitedate, 'White Ball', lastdate)
third_megacountdate = datelastdrawn(third_megacount, third_megadate, 'Mega Ball', lastdate)

third_mega_sorted = third_megacountdate.sort_values(by = 'Days Since Last Drawing')
third_white_sorted = third_whitecountdate.sort_values(by = 'Days Since Last Drawing')

### Mega Millions Frequency 10/31/2017 - 05/01/2020

In [79]:
fourth_df = mega_fourth.copy()

fourthdf = split_cols(fourth_df, 'First', 'Second', 'fourth', 'Fourth', 'Fifth')
fourth_mega = megaball(fourthdf, 2)
fourth_white = whiteball(fourthdf, 'White Ball')
fourth_megacount = ballcount(fourth_mega, 'Mega Ball', 'Mega Count')
fourth_whitecount = ballcount(fourth_white, 'White Ball', 'White Count')

fourth_whitedate = pd.DataFrame(checkdate(fourth_white).items(), columns=['White Ball', 'Date'])
fourth_megadate = pd.DataFrame(checkdate(fourth_mega).items(), columns=['Mega Ball', 'Date'])

fourth_whitecountdate = datelastdrawn(fourth_whitecount, fourth_whitedate, 'White Ball', 'Now')
fourth_megacountdate = datelastdrawn(fourth_megacount, fourth_megadate, 'Mega Ball', 'Now')

fourth_mega_sorted = fourth_megacountdate.sort_values(by = 'Days Since Last Drawing')
fourth_white_sorted = fourth_whitecountdate.sort_values(by = 'Days Since Last Drawing')

fourth_whitecountdate.head()
fourth_megacountdate.head()

,Mega Ball,Mega Count,Date,Days Since Last Drawing
0,11,15,2020-01-14,129 days
1,22,14,2020-03-13,70 days
2,20,14,2020-04-07,45 days
3,9,14,2020-01-17,126 days
4,10,14,2020-02-04,108 days
